<a href="https://colab.research.google.com/github/DeepLearningInterpreter/occlusion_experiments/blob/master/colab_notebooks/training_and_evaluation/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Training a gun detector


##Introduction
With this notebook a gun detector can be trained. 

It is advisable to save the checkoint (*.ckpt*) files this notebook outputs, so that the training progress will not be lost. For information about storing and loading external data with colab see: https://colab.research.google.com/notebooks/io.ipynb. From the checkpoint files training can be resumed at any time.



##Choose a Configuration to train


In the following codeblock it can be specified:

*  What the name of the model is you want to train.
*  For how many iterations you want to train the model.

The name of the model should match the name of the directory in the "/content/occlusion_experiments/main_content/multitude_of_possible_detectors/training/" folder where the "pipeline.config" file is stored.


In [0]:
which_model = "FRCNN_ext_lr3" #the name of the model
NUM_TRAIN_STEPS = 75000 #the number of desired training iterations

PIPELINE_CONFIG="/content/occlusion_experiments/main_content/multitude_of_possible_detectors/training/" + which_model + "/pipeline.config"
MODEL_DIR="/content/occlusion_experiments/main_content/multitude_of_possible_detectors/training/" + which_model

---------------------------------------------------------------------------
##Cloning the GitHub repository to the cloud server. 



In [0]:
#Downloading and installing git lfs
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!sudo apt-get install git-lfs
!git lfs install

Detected operating system as Ubuntu/bionic.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 72 not upgraded.
Need to get 4,931 kB of archives.
After this operation, 12.3 MB of additional disk space will be used.
Get:1 https://packagecloud.io/github/git-lfs/ubuntu bionic/main amd64 git-lfs amd64 2.6.1 [4,931 kB]
Fetched 4,931 kB in 1s (3,859 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/per

In [0]:
#This may take a while.

#Cloning repository. The exclude flag indicates that large files from the 
#"frozen_models for detection" and "evaluation_outcomes" subdirectories should 
#not be downloaded
!git lfs clone https://github.com/DeepLearningInterpreter/occlusion_experiments.git --exclude="occlusion_experiments/main_content/multitude_of_possible_detectors/frozen_models_for_detection, occlusion_experiments/main_content/multitude_of_possible_detectors/evaluation_outcomes"

          with new flags from 'git clone'

'git clone' has been updated in upstream Git to have comparable
speeds to 'git lfs clone'.
Cloning into 'occlusion_experiments'...
remote: Enumerating objects: 152, done.
remote: Counting objects: 100% (152/152), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 2150 (delta 96), reused 97 (delta 48), pack-reused 1998
Receiving objects: 100% (2150/2150), 185.83 MiB | 12.72 MiB/s, done.
Resolving deltas: 100% (373/373), done.
Checking out files: 100% (2204/2204), done.


In [0]:
import os
os.chdir("/content/occlusion_experiments/TF_object_detection_API_modified")
os.getcwd()

'/content/occlusion_experiments/TF_object_detection_API_modified'

In [0]:
%%capture
os.environ['PYTHONPATH'] += ":/content/occlusion_experiments/TF_object_detection_API_modified/slim"
!python setup.py install

In [0]:
!pip install pycocotools

In [0]:
import os
os.chdir('/content/occlusion_experiments/TF_object_detection_API_modified/object_detection')
os.getcwd()

'/content/occlusion_experiments/TF_object_detection_API_modified/object_detection'



---------------------------------------------------------------------------------------------------------------------
##Training begins


The model will automatically pick up training from the last saved checkpoint.

In [0]:
!python model_main.py \
    --pipeline_config_path=$PIPELINE_CONFIG \
    --model_dir=$MODEL_DIR \
    --num_train_steps=$NUM_TRAIN_STEPS \
    --sample_1_of_n_eval_examples=1 \
    --alsologtostderr

W0124 16:24:25.342016 139822863603584 tf_logging.py:125] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: 75000
I0124 16:24:25.342341 139822863603584 tf_logging.py:115] Maybe overwriting train_steps: 75000
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I0124 16:24:25.342494 139822863603584 tf_logging.py:115] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0124 16:24:25.342635 139822863603584 tf_logging.py:115] Maybe overwriting eval_num_epochs: 1
INFO:tensorflow:Maybe overwriting load_pretrained: True
I0124 16:24:25.342765 139822863603584 tf_logging.py:115] Maybe overwriting load_pretrained: True
INFO:tensorflow:Ignoring config override key: load_pretrained
I0124 16:24:25.342898 139822863603584 tf_logging.py:115] Ignoring config override key: load_pretrained
W0124 16:24:25.343552 139822863603584 tf_logging.py:125] Expected number of evaluation epochs is 1, bu


For changing the checkpoint saving frequency and changing the printing frequency adjust the arguments given to the RunConfig object in the model_main.py file. To influence the minimum time between evaluations change the throttle_secs argument passed to tf.estimator.EvalSpec() in the model_lib.py function. For reference check out: https://github.com/tensorflow/models/issues/5139#issuecomment-418963839. 